## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-23-18-53-29 +0000,bbc,Typhoon bursts Taiwan lake as strongest storm...,https://www.bbc.com/news/articles/c3rvjj7dp7ro...
1,2025-09-23-18-53-10 +0000,nyt,"Zubeen Garg, Singer of Hit Bollywood Song ‘Ya ...",https://www.nytimes.com/2025/09/22/world/asia/...
2,2025-09-23-18-53-00 +0000,wsj,"Sinclair, Nexstar Won’t Air Jimmy Kimmel’s Lat...",https://www.wsj.com/business/media/kimmels-lat...
3,2025-09-23-18-51-18 +0000,nyt,Powell Stresses ‘Challenging Situation’ for Fe...,https://www.nytimes.com/2025/09/23/business/je...
4,2025-09-23-18-50-58 +0000,nyt,"Burkina Faso, Mali and Niger Leaving Internati...",https://www.nytimes.com/2025/09/23/world/afric...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2324/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
44,trump,88
23,kimmel,21
22,jimmy,18
359,palestinian,15
43,president,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
5,2025-09-23-18-50-00 +0000,wsj,President Trump attacked global immigration an...,https://www.wsj.com/politics/trump-says-migrat...,163
233,2025-09-23-03-02-00 +0000,wsj,In labeling Tylenol use in pregnant women as a...,https://www.wsj.com/politics/in-targeting-a-co...,158
337,2025-09-22-20-35-04 +0000,nyt,Jimmy Kimmel’s Suspension Showed How Trump Ech...,https://www.nytimes.com/2025/09/22/business/ch...,145
18,2025-09-23-18-26-27 +0000,nypost,Trump supports NATO countries shooting down Ru...,https://nypost.com/2025/09/23/us-news/trump-su...,138
290,2025-09-22-22-45-00 +0000,wsj,Here’s what to know about Tylenol use during p...,https://www.wsj.com/politics/policy/tylenol-pr...,133


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
5,163,2025-09-23-18-50-00 +0000,wsj,President Trump attacked global immigration an...,https://www.wsj.com/politics/trump-says-migrat...
350,93,2025-09-22-19-43-36 +0000,nypost,Jimmy Kimmel set for return to ABC after backl...,https://nypost.com/2025/09/22/business/jimmy-k...
233,55,2025-09-23-03-02-00 +0000,wsj,In labeling Tylenol use in pregnant women as a...,https://www.wsj.com/politics/in-targeting-a-co...
34,45,2025-09-23-17-53-00 +0000,wsj,Fed Chair Jerome Powell said he judged the Fed...,https://www.wsj.com/economy/central-banking/fe...
18,41,2025-09-23-18-26-27 +0000,nypost,Trump supports NATO countries shooting down Ru...,https://nypost.com/2025/09/23/us-news/trump-su...
181,41,2025-09-23-10-00-00 +0000,nypost,New bill would crack down on bureaucrats worki...,https://nypost.com/2025/09/23/us-news/new-bill...
306,38,2025-09-22-21-47-05 +0000,nyt,U.S. allies’ recognition of a Palestinian stat...,https://www.nytimes.com/live/2025/09/22/world/...
71,36,2025-09-23-16-52-00 +0000,wsj,The Secret Service said it dismantled a networ...,https://www.wsj.com/politics/national-security...
75,34,2025-09-23-16-39-43 +0000,nypost,Kamala Harris rips Disney over Jimmy Kimmel su...,https://nypost.com/2025/09/23/business/kamala-...
117,33,2025-09-23-14-45-01 +0000,nypost,D4vd released creepy music video dragging body...,https://nypost.com/2025/09/23/us-news/d4vd-mad...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
